In [31]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
# Load training data
training = pd.read_csv("Training_set.csv", usecols=['SO2', 'CO', 'Proximity_to_Industrial_Areas', 'Air Quality'])
training_array = training[['SO2', 'CO', 'Proximity_to_Industrial_Areas']].to_numpy()
category_array = training['Air Quality'].to_numpy()


#Load validating data
validating= pd.read_csv("Validating_set.csv", usecols= ['SO2', 'CO', 'Proximity_to_Industrial_Areas', 'Air Quality'])
validating_array= validating[['SO2', 'CO', 'Proximity_to_Industrial_Areas']].to_numpy()
valcat_array= validating['Air Quality'].to_numpy()

#Load testing data
testing= pd.read_csv("Test_set.csv", usecols= ['SO2', 'CO', 'Proximity_to_Industrial_Areas', 'Air Quality'])
test_array= testing[['SO2', 'CO', 'Proximity_to_Industrial_Areas']].to_numpy()
testcat_array= testing['Air Quality'].to_numpy()


# Minkowski distance function (p=3)
def minkowski_distance(x, y, p=3):
    sum=0
    for i in range(len(x)):
        sum += abs(x[i] - y[i])**p
    return sum ** (1 / p)

# Function to categorize a point
def categorize(point):
    for i in range(len(training_array)):
        if np.array_equal(training_array[i], point):
            return category_array[i]
    return None

# K-Nearest Neighbors (KNN) function
def KNN(trainSet, trainLabels, point, k):
    distances = []

    # Calculate distances between the point and all training data
    for i in range(len(trainSet)):
        dist = minkowski_distance(trainSet[i], point)
        #add distances to the list
        distances.append((dist, trainLabels[i]))

    # Sort distances in ascending order and select top k
    distances.sort()
    top_k_distances = distances[:k]

    # Count occurrences of each category in the top k
    counts = {'Good': 0, 'Moderate': 0, 'Poor': 0, 'Hazardous': 0}
    # donot need distance value in the loop so use _ to ignore
    for _, category in top_k_distances:
        counts[category] += 1

    # Determine the category with the highest count

    #use key= counts.get to retrieve the value for a given key
    return max(counts, key=counts.get)

# Predict function for validation or testing set based on training set
def predict(valSet, trainSet, trainLabels, k):
    val_pred = []
    for i in range(len(valSet)):
        pred = KNN(trainSet, trainLabels, valSet[i], k)
        val_pred.append(pred)
    return val_pred

# Perform prediction on the validation and test sets
k = 10
val_pred = predict(validating_array, training_array, category_array, k)
test_pred = predict(test_array, training_array, category_array, k)

# Calculate accuracy of validation set
accuracy_val = accuracy_score(valcat_array, val_pred)
print("Validation accuracy: ", accuracy_val)

# Calculate accuracy of test set
accuracy_test = accuracy_score(testcat_array, test_pred)
print("Testing accuracy: ", accuracy_test)

Validation accuracy:  0.8775
Testing accuracy:  0.87
